In [16]:
import pandas as pd
from transformers import pipeline
import os
import statistics
import re
import spacy
from collections import Counter
from rapidfuzz import fuzz, process

In [2]:
#load the models
resolution_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
fineautomatabertweet = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

Device set to use cpu
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cpu


In [3]:
def extract_conversation_summary(file_path):
    """
    Extracts relevant lines from a conversation to summarise resolution.
    Focuses on Member lines.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    member_lines = [line.strip().split("Member: ", 1)[1] 
                              for line in lines if line.startswith("Member:")]
    
    return " ".join(member_lines) if member_lines else ""


def extract_member_lines(file_path):
    """
    Makes a dataframe of the Member lines.
    """
    member_lines = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("Member: "):
                # Append the line without the "Member: " part to the list
                member_lines.append(line[len("Member: "):].strip())
    df = pd.DataFrame(member_lines, columns=['Member_Line'])
    return df


def predict_resolution(file_path):
    """
    Predicts whether the issue in a conversation was resolved or not.
    """
    labels = ["resolved", "not resolved"]
    
    conversation_summary = extract_conversation_summary(file_path)
    if not conversation_summary:
        return "unknown"  # cases where no relevant data is available
    
    # use the resolution classifier
    result = resolution_classifier(conversation_summary, labels)
    return result["labels"][0]  # return the highest score


def predict_sentiment(file_path):
    """
    Given a path to a transcript text file, predict the sentiment of the caller.
    """    
    line_sentiment=[]
    member_lines = extract_member_lines(file_path)
    
    for i in range(len(member_lines)):
        line_result=fineautomatabertweet(member_lines['Member_Line'].iloc[i])
        line_score=line_result[0]['score']
    
        if line_score > 0.9:
            line_sentiment.append(line_result[0]['label'])

    targets = ["POS", "NEG", "NEU"]
    
    # Check for presence
    found = [item for item in targets if item in line_sentiment]
    mapping = {"POS": "positive", "NEU": "neutral", "NEG": "negative"}
    line_sentiment = [mapping.get(value, value) for value in line_sentiment]
    
    if not line_sentiment:
        return "neutral" # default to neutral
    else:    
        return statistics.mode(line_sentiment)

def extract_lines(file_path):
    """
    Extracts the conversation lines from the given transcript file.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()      
    return lines

def analyse_transcript(file_path):
    """
    Given a path to a transcript text file, predict both the sentiment of the caller and whether the issue was resolved or not.
    """
    sentiment_prediction = predict_sentiment(file_path)
    resolution_prediction = predict_resolution(file_path)

    return file_path, sentiment_prediction, resolution_prediction

In [4]:
directory = "transcripts/"  # directory containing call transcripts to be analysed

analysis_results = []

for file in os.listdir(directory):
    filename, sent, res = analyse_transcript(directory+file)
    analysis_results.append({'filename': filename, 'sentiment': sent, 'resolution': res})

In [5]:
df = pd.DataFrame(analysis_results)  # save the results in a dataframe

In [9]:
# make separate dataframes of the calls that were resolved and not resolved
unresolved_df = df[df['resolution'] == 'not resolved']
resolved_df = df[df['resolution'] == 'resolved']

In [10]:
unresolved_transcripts = unresolved_df['filename'].tolist()
resolved_transcripts = resolved_df['filename'].tolist()

In [11]:
nlp = spacy.load("en_core_web_sm") # load the natural language processing library, goes into prepocess_first_line

In [12]:
# made a few custom stop words and irrelevant patterns to filter out - add to this as you like
custom_stop_words = {"member", "support", "customer", "call", "hi", "hello", "thanks", "thank", "you"}
irrelevant_patterns = [r"^Member:\s*", r"MEM\d+", r"^\w+:\s+"]

In [13]:
def preprocess_first_line(file_path):
    """
    Given a file path, extracts the first line and preprocesses it for NLP analysis.
    First line is usually where the issue is stated, so this is the best place to focus the analysis.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        # read only the first line and clean it
        line = f.readline().strip()
        for pattern in irrelevant_patterns:
            line = re.sub(pattern, '', line, flags=re.IGNORECASE)
    # use NLP library defined above
    doc = nlp(line)
    filtered_tokens = [
        token.lemma_ for token in doc 
        if token.pos_ in {"NOUN", "VERB"} and token.text.lower() not in custom_stop_words
    ]
    return " ".join(filtered_tokens)

def consolidate_phrases(phrases, similarity_threshold=85):
    """
    Given a list of phrases, do a fuzzy match to find similarities.
    """
    unique_phrases = []
    phrase_counts = Counter(phrases)

    for phrase in phrase_counts:
        # match the phrase to the most similar one already in unique_phrases
        result = process.extractOne(phrase, unique_phrases, scorer=fuzz.ratio)
        
        # check if a match was found
        if result is not None:
            match, score, _ = result
            if score >= similarity_threshold:
                # if a similar phrase exists, update its count
                existing_index = unique_phrases.index(match)
                unique_phrases[existing_index] = match  # Keep the original match
                phrase_counts[match] += phrase_counts[phrase]
            else:
                # otherwise, add as a new unique phrase
                unique_phrases.append(phrase)
        else:
            # if no match, add as a new unique phrase
            unique_phrases.append(phrase)

    # return phrases sorted by occurrence
    consolidated_counts = [(phrase, phrase_counts[phrase]) for phrase in unique_phrases]
    return sorted(consolidated_counts, key=lambda x: x[1], reverse=True)

def analyze_common_themes_with_consolidation(file_list):
    """
    Given a list of support call transcripts, determine whether there are common themes in the opening of the conversation.
    """
    all_phrases = []

    for file_path in file_list:
        preprocessed_line = preprocess_first_line(file_path)
        if preprocessed_line:  # only include non-empty lines
            all_phrases.append(preprocessed_line)
    
    # consolidate similar phrases
    common_themes = consolidate_phrases(all_phrases)

    # calculate percentage of calls for each theme
    total_calls = len(file_list)
    
    # display themes by percentage of calls
    for theme, count in common_themes:
        percentage = (count / total_calls) * 100
        print(f"{theme}: {percentage:.2f}% of calls")

In [17]:
# run the analysis on the transcripts of unresolved calls
analyze_common_themes_with_consolidation(unresolved_transcripts)   # looks like mostly tech side of things

have trouble register log service account: 19.40% of calls
call claim receive cover name: 5.97% of calls
call schedule appointment specialist name: 5.97% of calls
call visit doctor charge copay policy say suppose: 4.48% of calls
call have trouble log service account: 4.48% of calls
call issue account have trouble register log: 4.48% of calls
call claim deny name: 4.48% of calls
have trouble log account website: 4.48% of calls
call claim receive service tell policy cover cover policy: 2.99% of calls
have trouble register log service account try time keep get error message: 2.99% of calls
call visit doctor office charge copay think policy have copay service: 2.99% of calls
try register log service account have issue: 2.99% of calls
call claim receive service believe deny error: 1.49% of calls
call claim deny reason give policy cover service cover: 1.49% of calls
call get case pre - surgery need name: 1.49% of calls
call copay doctor visit charge policy say pay care visit: 1.49% of calls


In [18]:
# run the analysis on the transcripts of resolved calls
analyze_common_themes_with_consolidation(resolved_transcripts)  # we see that the most common call issue from the unresolved transcripts 
                                                                # occurs at only half the rate in the resolved calls. Looks like we need 
                                                                # to look for issues in the tech side of things

call schedule appointment specialist name: 21.80% of calls
call get case pre - name: 10.53% of calls
have trouble register log service account: 7.52% of calls
call get case pre - procedure name: 6.77% of calls
call doctor visit charge copay policy say have pay service: 6.02% of calls
call claim receive service name: 6.02% of calls
call request pre - authorization procedure name: 5.26% of calls
call visit doctor office charge copay think policy have copay service: 3.01% of calls
call bill doctor visit charge copay think suppose accord policy: 2.26% of calls
call claim receive service believe deny policy cover: 2.26% of calls
call request case pre - authorization procedure need undergo name: 1.50% of calls
call claim receive letter state service cover policy switch policy cover: 1.50% of calls
call doctor visit charge copay policy say suppose copay: 1.50% of calls
call service account have trouble register log: 1.50% of calls
need schedule appointment specialist help: 1.50% of calls
call